In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, lower, col, when

import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder\
    .appName("Nettoyage de Marketing")\
    .enableHiveSupport()\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 09:17:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

df_marketing = spark.sql("SELECT * FROM marketing_ext")
df_marketing = df_marketing.filter(df_marketing['sexe'] != 'sexe') # A GERER DANS L'IMPORT DE HIVE ???

In [4]:
df_marketing.show(n=1000)

+---+----+------+------------------+---------------+---------------+
|age|sexe|  taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|
+---+----+------+------------------+---------------+---------------+
| 21|   F|1396.0|       C�libataire|              0|          false|
| 35|   M| 223.0|       C�libataire|              0|          false|
| 48|   M| 401.0|       C�libataire|              0|          false|
| 26|   F| 420.0|         En Couple|              3|           true|
| 80|   M| 530.0|         En Couple|              3|          false|
| 27|   F| 153.0|         En Couple|              2|          false|
| 59|   F| 572.0|         En Couple|              2|          false|
| 43|   F| 431.0|       C�libataire|              0|          false|
| 64|   M| 559.0|       C�libataire|              0|          false|
| 22|   M| 154.0|         En Couple|              1|          false|
| 79|   F| 981.0|         En Couple|              2|          false|
| 55|   M| 588.0|       C�libatair

### Remarque 001
- Le symbole "�" apparaît dans la colonne **situationfamiliale**.
- Des majuscule sont dans la colonne **situationfamiliale**.


In [5]:
#Le symbole "�" apparaît dans la colonne situationfamiliale.
df_marketing = df_marketing.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "�", "e"))
df_marketing = df_marketing.withColumn("situationfamiliale", lower(col("situationfamiliale")))

In [6]:
df_marketing.show(n=1000)
df_marketing.printSchema()

+---+----+------+------------------+---------------+---------------+
|age|sexe|  taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|
+---+----+------+------------------+---------------+---------------+
| 21|   F|1396.0|       celibataire|              0|          false|
| 35|   M| 223.0|       celibataire|              0|          false|
| 48|   M| 401.0|       celibataire|              0|          false|
| 26|   F| 420.0|         en couple|              3|           true|
| 80|   M| 530.0|         en couple|              3|          false|
| 27|   F| 153.0|         en couple|              2|          false|
| 59|   F| 572.0|         en couple|              2|          false|
| 43|   F| 431.0|       celibataire|              0|          false|
| 64|   M| 559.0|       celibataire|              0|          false|
| 22|   M| 154.0|         en couple|              1|          false|
| 79|   F| 981.0|         en couple|              2|          false|
| 55|   M| 588.0|       celibatair

## Vérification du domaines de valeurs

In [7]:
age_inf = 18
age_supp = 84

result = df_marketing.filter(df_marketing['age'] < age_inf).count()
print(f'Age inférieure à {age_inf} ans: {result}')

result = df_marketing.filter(df_marketing['age'] > age_supp).count()
print(f'Age supérieure à {age_supp} ans: {result}')

Age inférieure à 18 ans: 0
Age supérieure à 84 ans: 0


In [8]:
result = df_marketing.filter((df_marketing['sexe'] != 'F') & (df_marketing['sexe'] != 'M')).count()
print(f'Sexe différent de \'F\' ou \'M\': {result}')

Sexe différent de 'F' ou 'M': 0


In [9]:
taux_inf = 544
taux_supp = 74185

# Total des lignes
total = df_marketing.count()

# Calculer et afficher les résultats
result_inf = df_marketing.filter(df_marketing['taux'] < taux_inf).count()
percent_inf = (result_inf / total) * 100
print(f'Taux inférieure à {taux_inf}: {result_inf} ({percent_inf:.2f}%)')

result_supp = df_marketing.filter(df_marketing['taux'] > taux_supp).count()
percent_supp = (result_supp / total) * 100
print(f'Taux supérieure à {taux_supp}: {result_supp} ({percent_supp:.2f}%)')

Taux inférieure à 544: 11 (55.00%)
Taux supérieure à 74185: 0 (0.00%)


In [10]:
list_situation_familiale_domain_value = [
    'celibataire',
    'en couple',
    'divorce',
    'marie(e)',
    'seul',
    'seule'
]

result = df_marketing.filter(~df_marketing['situationfamiliale'].isin(list_situation_familiale_domain_value)).count()
print(f"Nombre d'enregistrements avec 'situation_familiale' valide : {result}")

Nombre d'enregistrements avec 'situation_familiale' valide : 0


In [11]:
enfant_inf = 0
enfant_supp = 4

result = df_marketing.filter(df_marketing['nbenfantacharge'] < enfant_inf).count()
print(f'Nombre d\'enfant à charge inférieure à {enfant_inf}: {result}')

result = df_marketing.filter(df_marketing['nbenfantacharge'] > enfant_supp).count()
print(f'Nombre d\'enfant à charge supérieure à {enfant_supp}: {result}')

Nombre d'enfant à charge inférieure à 0: 0
Nombre d'enfant à charge supérieure à 4: 0


### Réorganisation du dataframe marketing

In [12]:
df_marketing = df_marketing.withColumn(
    "taux_eligible",
    when((df_marketing["taux"] < taux_inf) | (df_marketing["taux"] > taux_supp), True).otherwise(False)
)

df_marketing.show()

+---+----+------+------------------+---------------+---------------+-------------+
|age|sexe|  taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|taux_eligible|
+---+----+------+------------------+---------------+---------------+-------------+
| 21|   F|1396.0|       celibataire|              0|          false|        false|
| 35|   M| 223.0|       celibataire|              0|          false|         true|
| 48|   M| 401.0|       celibataire|              0|          false|         true|
| 26|   F| 420.0|         en couple|              3|           true|         true|
| 80|   M| 530.0|         en couple|              3|          false|         true|
| 27|   F| 153.0|         en couple|              2|          false|         true|
| 59|   F| 572.0|         en couple|              2|          false|        false|
| 43|   F| 431.0|       celibataire|              0|          false|         true|
| 64|   M| 559.0|       celibataire|              0|          false|        false|
| 22

## Création de la table

In [14]:
# Nom de la table cible
table_name = "marketing"

table_exists = spark._jsparkSession.catalog().tableExists("concessionnaire", table_name)

if not table_exists:
    print('Création et enregistrement de la table "marketing"')
    df_marketing.write.saveAsTable(table_name)
else:
    print('Enregistrement dans la table "marketing"')
    df_marketing.write.mode("overwrite").saveAsTable(table_name)

Création et enregistrement de la table "marketing"
